In [1]:
import sqlite3
import pymysql
pymysql.install_as_MySQLdb()
import MySQLdb

<div class="alert alert-block alert-success"><font size="+3"><b>
Getting Started with SQL
</b></font></div>

In this article, we go through some of the basic functions of SQL Server. For the full tutorial please see [SQL Server technical documentation](https://docs.microsoft.com/en-us/sql/sql-server/?view=sql-server-ver15). Since we are working on a Jupyter Notebook, all of SQL commands here complied through [SQLite](https://www.sqlite.org/index.html) and [PyMySQL](https://pymysql.readthedocs.io/en/latest/).

# Working with Tables in SQLite

## Creating a Table
We can create a new table in a using SQL with the following syntax,

```SQL
CREATE TABLE table_name (
column1 datatype,
column2 datatype,
column3 datatype,
   ....
);```

However, we can use this syntax in Python as well. In doing so, we need to use the **sqlite3** package. Let's create ``mydatabase.db``, and enter the all-time top goal scorers of the premier from this [website](https://www.premierleague.com/stats/top/players/goals?co=1&se=-1&co=1&cl=-1&iso=-1&po=-1?se=-1).

To create *Players* table we can use the following SQL command

```SQL
CREATE TABLE Players ( 
Rank INTEGER PRIMARY KEY, 
First_Name VARCHAR(20), 
Last_Name VARCHAR(30), 
Nationality CHAR(10), 
Goals INTEGER,
Birth_Date DATE
);
```

In python, this can be done using

In [2]:
# creating *mydatabase.db* in the current directory
connection = sqlite3.connect("mydatabase.db")

cursor = connection.cursor()

# DROP IF EXISTS is only available
_ = cursor.execute("""DROP TABLE IF EXISTS Players""")

# SQL table syntax
sql_command = """
CREATE TABLE Players ( 
    Rank INTEGER PRIMARY KEY, 
    First_Name VARCHAR(20), 
    Last_Name VARCHAR(30), 
    Nationality CHAR(10), 
    Goals INTEGER,
    Birth_Date DATE
    );"""

# running that syntax
_ = cursor.execute(sql_command)

## Inserting a table row

For example, to enter a row of a player data, we can use the following SQL syntax

```SQL
INSERT INTO Players (Rank, First_Name, Last_Name, Nationality, Goals, Birth_Date)
VALUES (Null, "Alan", "Shearer", "England", 260,"1970-08-13");
```

In Python, this can be done as follows.

In [3]:
# entering a row
sql_command = """INSERT INTO Players (Rank, First_Name, Last_Name, Nationality, Goals, Birth_Date)
    VALUES (Null, "Alan", "Shearer", "England", 260,"1970-08-13");"""
cursor.execute(sql_command)

# Saving the changes
connection.commit()
# closing hte file
connection.close()

## Inserting multiple table rows

As you noticed, entering table rows by the previous command is not convenient, especially for entering a large number of rows. Therefore, we can enter the data as follows instead.

In [4]:
connection = sqlite3.connect("mydatabase.db")
cursor = connection.cursor()
# Data
Players_data = [ ("Wayne", "Rooney", "England", 208, "1985-10-24"),
               ("Andrew", "Cole", "England", 187, "1971-10-15"),
               ("Frank", "Lampard", "England", 177, "1978-06-20") ]
# string format
format_str = """INSERT INTO Players (Rank, First_Name, Last_Name, Nationality, Goals, Birth_Date)
VALUES (NULL, "{first}", "{last}", "{nationality}", "{goals}", "{birthdate}");"""
for p in Players_data:
    sql_command = format_str.format(first=p[0], last=p[1], nationality=p[2], goals = p[3], birthdate = p[4])
    cursor.execute(sql_command)
# Saving
connection.commit()
connection.close()

## Reading a table

Let's see what we have entered into the table. We can use a **for** lop to print all entered rows.

In [5]:
connection = sqlite3.connect("mydatabase.db")

cursor = connection.cursor()

cursor.execute("SELECT * FROM Players") 
print("All rows:")
result = cursor.fetchall() 
for row in result:
    print(row)
del row

Alternatively, we can print a row.

In [6]:
cursor.execute("SELECT * FROM Players") 
print("One row:")
row = cursor.fetchone() 
print(row)

We could also print all using a **while** loop as well

In [7]:
cursor.execute("SELECT * FROM Players")
row = cursor.fetchone()
while row is not None:
  print(row)
  row = cursor.fetchone()

Moreover, we can define the following function for showing the table (for later uses).

In [8]:
def Show_Table():
    connection = sqlite3.connect("mydatabase.db")
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM Players") 
    print("All rows:")
    result = cursor.fetchall() 
    for row in result:
        print(row)
    del row

### Updating a table

Assume that we are going to enter the following row in our table.

| Last Name | First Name | Nationality | Goals | Birthdate |
|-----------|------------|-------------|-------|------------|
| Thierry | Henry | France | 175 | 1977-08-17 |

In [9]:
connection = sqlite3.connect("mydatabase.db")

cursor = connection.cursor()

Players_data = [ ("Thierry", "Henry", "France", 180, "1977-08-17")]
               
format_str = """INSERT INTO Players (Rank, First_Name, Last_Name, Nationality, Goals, Birth_Date)
VALUES (NULL, "{first}", "{last}", "{nationality}", "{goals}", "{birthdate}");"""

for p in Players_data:
    sql_command = format_str.format(first=p[0], last=p[1], nationality=p[2], goals = p[3], birthdate = p[4])
    cursor.execute(sql_command)
    
connection.commit()

connection.close()

In [10]:
Show_Table()

All rows:
(1, 'Alan', 'Shearer', 'England', 260, '1970-08-13')
(2, 'Wayne', 'Rooney', 'England', 208, '1985-10-24')
(3, 'Andrew', 'Cole', 'England', 187, '1971-10-15')
(4, 'Frank', 'Lampard', 'England', 177, '1978-06-20')
(5, 'Thierry', 'Henry', 'France', 180, '1977-08-17')


However, we entered another value by mistake for goal numbers.

| Last Name | First Name | Nationality | Goals | Birthdate |
|-----------|------------|-------------|-------|------------|
| Thierry | Henry | France | 180 | 1977-08-17 |

 In this case, we can update the table as follows
 
```SQL
UPDATE Players
SET goals =175
WHERE Rank=5;
```

In [11]:
connection = sqlite3.connect("mydatabase.db")
cursor = connection.cursor()
# updating a row
cursor.execute("""UPDATE Players
    SET goals =175
    WHERE Rank=5;""")
   
connection.commit()
connection.close()

As a result, the table has been updated as follows,

In [12]:
Show_Table()

All rows:
(1, 'Alan', 'Shearer', 'England', 260, '1970-08-13')
(2, 'Wayne', 'Rooney', 'England', 208, '1985-10-24')
(3, 'Andrew', 'Cole', 'England', 187, '1971-10-15')
(4, 'Frank', 'Lampard', 'England', 177, '1978-06-20')
(5, 'Thierry', 'Henry', 'France', 175, '1977-08-17')


## Copying a Table

Syntax:

```SQL
CREATE TABLE new_table AS SELECT * FROM original_table;```

In [13]:
connection = sqlite3.connect("mydatabase.db")
cursor = connection.cursor()
cursor.execute("CREATE TABLE new_Players AS SELECT * FROM Players;")
connection.commit()
connection.close()

## Deleting records in a table

### Deleting a row

syntax
```SQL
DELETE FROM table_name WHERE condition;```

In [14]:
connection = sqlite3.connect("mydatabase.db")
cursor = connection.cursor()
cursor.execute("""DELETE FROM new_Players WHERE Last_Name="Henry";""")
connection.commit()
connection.close()

Showing the *new_Player* table

In [15]:
connection = sqlite3.connect("mydatabase.db")
cursor = connection.cursor()
cursor.execute("SELECT * FROM new_Players") 
print("All rows:")
result = cursor.fetchall() 
for row in result:
    print(row)
del row

All rows:
(1, 'Alan', 'Shearer', 'England', 260, '1970-08-13')
(2, 'Wayne', 'Rooney', 'England', 208, '1985-10-24')
(3, 'Andrew', 'Cole', 'England', 187, '1971-10-15')
(4, 'Frank', 'Lampard', 'England', 177, '1978-06-20')


As can be seen, the last row is deleted.

### Deleting a table

syntax
```SQL
DROP TABLE table_name;```

In [16]:
connection = sqlite3.connect("mydatabase.db")
cursor = connection.cursor()
cursor.execute("DROP TABLE new_Players;")
connection.commit()
connection.close()